# Instalar dependencias
---

In [1]:
!( [ -d computer-vision-cards ] && cd computer-vision-cards/ && git pull origin main ) || git clone https://github.com/amaranto/computer-vision-cards.git

Cloning into 'computer-vision-cards'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 1), reused 14 (delta 0), pack-reused 0
Receiving objects: 100% (18/18), 15.68 KiB | 2.61 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [4]:
! pip install unicode unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.2 MB/s eta 0:00:00


# Dataset
---

<p> Se utilizo gdown y un script de bash para descargar todos los datasets subidos al formulario solicitado del ejecicio 1 y se unificaron en una sola carpeta, la cual fue subido a mi drive personal.</p>
<p> Luego se desarrollo un script de python, el cual sigue los siguientes pasos :<p>


1.   Recorre todos los archivos de etiquetas tomando como base el nombre del archivo sin la extension .txt, renombra el nombre de los archivos .txt e imagenes que contengan acentos o caracteres extranos del dataset original.
2.   Si las etiquetas estan en formato polygon entonces las transforma a `x1,y1,x2,y2` y redimensiona las coordenadas a su tamano original.
3.   Luego utiliza cv2 para generar el bbox que se ajuste mas a las coordenadas definidas en el archivo de etiqueta.
4.   Guarda imagenes con los bboxes generados sobre las mismas para posterior verificacion manual en la carpeta `dataset/output/bbox_check_folder/img`
5.   Convierte las nuevas coordenadas a formato yolo con bbox y guarda los archivos en `dataset/output/bbox_check_folder/labels`
6.   Copia los nuevos archivos de etiquetas generados desde `dataset/output/bbox_check_folder/labels` al dataset original
7.   Genera el datset en formato yolo en `.datasets/yolo-format`, generando la carpeta de validacion y training respectivamente, utilizando los archivos del dataset original y los que fueron corregidos previamente.
8.   Se detecta que los archivos de `51705_micaela_pozzo_.*` estan mal etiquetados, los archivos `*.txt` no estan relacionados con las imagenes , por lo que se proceden a borrar manualmente con `rm -f ./computer-vision-cards/datasets/yolo-format/{images,labels}/{train,val}/51705_micaela_pozzo_** `.
</p>


In [5]:
!cd computer-vision-cards/ &&  python datasetHandler.py

Downloading...
From (original): https://drive.google.com/uc?id=1u7eRhAuarRWJNNMTsdRIh18fBL3Lhlka
From (redirected): https://drive.google.com/uc?id=1u7eRhAuarRWJNNMTsdRIh18fBL3Lhlka&confirm=t&uuid=8638b898-468c-4a70-bea8-d6b0886d7970
To: /content/computer-vision-cards/datasets/1u7eRhAuarRWJNNMTsdRIh18fBL3Lhlka.zip
100% 4.52G/4.52G [00:50<00:00, 90.4MB/s]
Found 1 folders but working on ds-todas-las-cartas
Processing ./datasets/dowloaded//ds-todas-las-cartas/aba2720d35f5ea68fae3f1edca09510b.png...
Processing ./datasets/dowloaded//ds-todas-las-cartas/71846_Brisa_Menescaldi_29.png...
Processing ./datasets/dowloaded//ds-todas-las-cartas/26f38499ae0ac746a12238844114ee2b.png...
Processing ./datasets/dowloaded//ds-todas-las-cartas/71846_Brisa_Menescaldi_06.png...
Processing ./datasets/dowloaded//ds-todas-las-cartas/d429816_cesar_donnarumma_01.png...
Processing ./datasets/dowloaded//ds-todas-las-cartas/F37605_florencia_fernandez_03.png...
Processing ./datasets/dowloaded//ds-todas-las-cartas/R-45

In [9]:
!rm -f ./computer-vision-cards/datasets/yolo-format/{images,labels}/{train,val}/51705_micaela_pozzo_*

# Augmentation
---
Se utiliza albumentation para aumentar el dataset generado, se utilizan rotaciones, aumento y variacion de brillo y contraste, redimension y perspectiva sobre el dataset generado.

Las imagenes son generadas sobre el dataset original, al igual que sus archivos de etiquetas, pero con el sufijo `_augmented_<id>`.

Algo importante a mencionar, es que al utilizar PIL, las imagenes tomadas con el celular en forma horizontal, invierten sus coordenadas x,y, por lo que se utiliza la opcion `ImageOps.exif_transpose(orig)`.

Guardamos aparte las imagenes generadas con los bboxes para verficacion en la carpeta `./datasets/augmentation-check` .


In [ ]:
!cd ./computer-vision-cards && python datasetAugmenter.py

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations
--------------------------------------------------
Image: ./datasets/yolo-format/images/train/37524_augusto_farias_20.jpg 0/745
/usr/local/lib/python3.10/dist-packages/albumentations/core/composition.py:156: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
Image: ./datasets/yolo-format/images/train/37524_augusto_farias_20.jpg Destiny:./datasets/yolo-format/images/train//37524_augusto_farias_20_augmented_0.png Label: ./datasets/yolo-format/labels/train//37524_augusto_farias_20_augmented_0.txt Aug: 0/8
Image: ./datasets/yolo-format/images/train/37524_augusto_farias_20.jpg Destiny:./datasets/yolo-format/images/train//37524_augusto_farias_20_augmented_1.png Label: ./datasets/yolo-format/labels/train//37524_augusto_farias_20_augmented_1.txt Aug: 1/8
Image: ./datasets/yolo-format/images/train/375

# Visualizacion del dataset Pre-entrenamiento
---
Se

In [ ]:
!cd ./computer-vision-cards && python datasetViewer.py

# Subimos el dataset generado y verificado a Google Drive
---

# Entrenamiento
---

Utilizamos YOLOv8 en su version m para Object Detection: `model=YOLO("yolov8m.yaml")`

*  Se utiliza GPU si esta disponible : `device=device`
*  Se utiliza hasta el 80% de la memoria para generar batches de entrenamiento: `batch=0.8`
*  Se espera hasta 20 epocas y si no hay mejoras se termina el entrenamiento : `patience=20`
* Se habilita para guardar los pesos de entrenamiento : `save=True`
* Se guarda cada 10 epocas: `save_period=10`
* No se resume para el primer entrenamiento : `resume=False`
* Se habilita la validaccion : `val=True`
* Se habilita la generacion de imagenes para graficar metricas y entender mejor el funionamiento y performance del modelo : `plots=True`
* Se utiliza resize de las imagenes para el entrenamiento de 640x640. Lo ideal seria aumentarlo por lo menos a 1024, pero debido la falta de recursos de hardware no fue posible, se duplica el uso de memoria:  `imgsz = 640`
* Se entrena 100 epocas `epochs=100`
* Se aumenta el peso de la clasificacion en la funcion de perdida para darle mas importancia a la deteccion de la clase correcta `cls=0.8`
* Se habilita la aumentaion de datos al entrenar : `augment=True`
* Se deshabilitan para la aumentacion : `shear` ya que no necesitamos que se corten las fotos porque hay bastante oclusion de las mismas en el dataset de etrenamiento y `bgr` porque el color para la deteccion del palo es importante.



In [ ]:
!cd ./computer-vision-cards && python train.py